In [30]:
import requests
import pandas as pd
import random
import logging
from langchain import PromptTemplate, LLMChain
from langchain.llms import Ollama
from langchain.text_splitter import CharacterTextSplitter
from datetime import datetime, timedelta

text_splitter = CharacterTextSplitter(separator=" ", chunk_size=1000, chunk_overlap=200)  # smaller chunks
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize the Ollama LLM through LangChain
llm = Ollama(model="llama3")

territories = [
    {"id": 4302105, "name": "Bento Gonçalves", "state": "RS"},
    {"id": 4303103, "name": "Cachoeira do Sul", "state": "RS"},
    {"id": 4303509, "name": "Camaquã", "state": "RS"},
    {"id": 4303905, "name": "Candelária", "state": "RS"},
    {"id": 4305108, "name": "Caxias do Sul", "state": "RS"},
    {"id": 4305207, "name": "Cerrito", "state": "RS"},
    {"id": 4306403, "name": "Dois Irmãos", "state": "RS"},
    {"id": 4307609, "name": "Estrela", "state": "RS"},
    {"id": 4309209, "name": "Gravataí", "state": "RS"},
    {"id": 4309605, "name": "Horizontina", "state": "RS"},
    {"id": 4311809, "name": "Marau", "state": "RS"},
    {"id": 4313904, "name": "Panambi", "state": "RS"},
    {"id": 4314902, "name": "Porto Alegre", "state": "RS"},
    {"id": 4316758, "name": "Santa Clara do Sul", "state": "RS"},
    {"id": 4317202, "name": "Santa Rosa", "state": "RS"},
    {"id": 4320800, "name": "Sobradinho", "state": "RS"},
    {"id": 4318705, "name": "São Francisco de Paula", "state": "RS"},
    {"id": 4318432, "name": "São João do Polêsine", "state": "RS"},
    {"id": 4321709, "name": "Três Arroios", "state": "RS"},
    {"id": 4322806, "name": "Vera Cruz", "state": "RS"},
]

# Step 1: Generate a random date between January 1, 2022, and August 31, 2024
def random_date(start_date, end_date):
    """Generate a random date between start_date and end_date."""
    random_days = random.randint(0, (end_date - start_date).days)
    return start_date + timedelta(days=random_days)

def random_date_range(start_year=2022, end_date="2024-08-31"):
    """Generate a valid random date range with start date before end date."""
    start_date = datetime(start_year, 1, 1)
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    random_start = random_date(start_date, end_date)
    random_end = random_date(random_start, end_date)
    return random_start.strftime("%Y-%m-%d"), random_end.strftime("%Y-%m-%d")

# Define the classification labels
class ClassificationEnum:
    NOT_FACT = "not-fact"
    FALSE = "false"
    MISLEADING = "misleading"
    UNSUSTAINABLE = "unsustainable"
    UNVERIFIABLE = "unverifiable"
    EXAGGERATED = "exaggerated"
    ARGUABLE = "arguable"
    TRUSTWORTHY_BUT = "trustworthy-but"
    TRUSTWORTHY = "trustworthy"

classification_labels = [
    ClassificationEnum.FALSE,
    ClassificationEnum.MISLEADING,
    ClassificationEnum.EXAGGERATED,
    ClassificationEnum.TRUSTWORTHY_BUT,
    ClassificationEnum.TRUSTWORTHY,
]

# Step 1: Function to query the API with territory_ids included
def query_gazettes(query=None, since=None, until=None, offset=0, size=10, territory_ids=None):
    url = "https://queridodiario.ok.org.br/api/gazettes"
    params = {
        "querystring": query,
        "published_since": since,
        "published_until": until,
        "offset": offset,
        "size": size,
        "territory_ids": territory_ids
    }
    
    logging.info(f"Querying gazettes with params: {params}")
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        logging.info("Gazettes retrieved successfully.")
        return response.json()
    else:
        logging.error(f"Failed to retrieve gazettes. Status code: {response.status_code}")
        return None

# Step 2: Function to download and read the text from gazette
def download_gazette_text(txt_url):
    logging.info(f"Downloading text from: {txt_url}")
    
    response = requests.get(txt_url)
    
    if response.status_code == 200:
        logging.info("Text downloaded successfully.")
        return response.text
    else:
        logging.error(f"Failed to download text from {txt_url}. Status code: {response.status_code}")
        return None

# Step 3: Function to use LLM for claim generation using LangChain
def generate_claims_from_gazette(text, num_claims=5):
    logging.info(f"Gerando {num_claims} afirmações a partir do texto do diário oficial.")
    
    # Define a prompt template in Portuguese
    prompt_template = PromptTemplate(
        input_variables=["text", "num_claims"],
        template=(
            """
            Extraia {num_claims} afirmações factuais potenciais do seguinte texto de um diário oficial municipal:\n\n{text}\n\n
            Apenas coloque no output as claims sem enumerar ou adicionar comentário sobre. O texto deve ser sempre em português.
            """
        ),
    )
    
    # Create an LLMChain
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Run the chain to generate claims
    claims_output = chain.run({"text": text, "num_claims": num_claims})
    
    # Log the output from the LLM
    logging.info(f"Afirmações geradas pelo LLM: {claims_output}")
    
    # Split the output into individual claims
    claims = [claim.strip() for claim in claims_output.split('\n') if claim.strip()]
    
    return claims


# Step 4: Generate classified claims with the new classification labels using LangChain
def generate_classified_claims(claims, metadata):
    logging.info("Gerando classificações para as afirmações.")
    
    dataset = []
    for claim in claims:
        classification = random.choice(classification_labels)
        
        # Optionally, modify the claim slightly based on the classification
        if classification != ClassificationEnum.TRUSTWORTHY:
            logging.info(f"Criando uma versão modificada da afirmação com a classificação: {classification}")
            
            prompt_template = PromptTemplate(
                input_variables=["claim", "classification"],
                template="""
                Você é um especialiasta em desinformação e seu papel é gerar um dataset sintético para treinar uma LLM checadora de fatos.
                Você vai receber informações de diários oficiais, encontrar afirmações e modificá-las para o dataset incorporando a classificação desejada.
                Sempre use a língua portuguesa para respostas.
                Não explique ou adicione qualquer formatção ou texto que não seja da afirmação modificada.
                Não escreva a afirmação original e como se transformou na modificada.
                
                <tarefa>Dada a afirmação: \"{claim}\", modifique-a para refletir a classificação \"{classification}\".</tarefa>
                """,
            )
            chain = LLMChain(llm=llm, prompt=prompt_template)
            claim = chain.run({"claim": claim, "classification": classification}).strip()
            logging.info(f"Afirmação modificada gerada: {claim}")
        
        # Combine the claim and classification with the metadata
        claim_data = {
            "claim": claim,
            "classification": classification,
            **metadata  # Add all metadata fields
        }
        dataset.append(claim_data)
    
    logging.info(f"Conjunto de dados gerado: {dataset}")
    return dataset

# Main function to create the synthetic dataset
def create_synthetic_dataset(query=None, territory=None, dates=None, num_gazettes=3, num_claims_per_gazette=2, num_random_chunks=10):
    logging.info("Starting synthetic dataset creation.")
    
    all_classified_claims = []
    
    for i in range(num_gazettes):
    # Generate random territories and dates
        random_date_range_values = random_date_range()
        dates = {"since": random_date_range_values[0], "until": random_date_range_values[1]}
        random_since = dates['since']
        random_until = dates['until']
    
        gazette_data = query_gazettes(since=random_since, until=random_until, size=num_gazettes, territory_ids=[territory["id"]])
        print(gazette_data)
        if gazette_data and 'gazettes' in gazette_data:
            total_gazettes = len(gazette_data['gazettes'])
            logging.info(f"Total gazettes to process: {total_gazettes}")
            
            all_classified_claims = []
            
            for idx, gazette in enumerate(gazette_data['gazettes'], start=1):
                txt_url = gazette.get('txt_url')
                gazette_url = gazette.get('url')
                date = gazette.get('date')
                territory_id = gazette.get('territory_id')
                territory_name = gazette.get('territory_name')
                state_code = gazette.get('state_code')
                is_extra_edition = gazette.get('is_extra_edition')
                
                if txt_url:
                    logging.info(f"Processing gazette {idx}/{total_gazettes}: {txt_url}")
                    
                    text = download_gazette_text(txt_url)
                    if text:
                        chunks = text_splitter.split_text(text)
                        logging.info(f"Text split into {len(chunks)} chunks for gazette {idx}.")
                        
                        # Select random chunks
                        selected_chunks = random.sample(chunks, min(num_random_chunks, len(chunks)))
                        
                        for chunk in selected_chunks:
                            claims = generate_claims_from_gazette(chunk, num_claims=num_claims_per_gazette)
                            
                            # Create metadata dictionary to pass along with the claims
                            metadata = {
                                "query": query,
                                "since": random_since,
                                "until": random_until,
                                "territory_id": territory_id,
                                "territory_name": territory_name,
                                "state_code": state_code,
                                "gazette_date": date,
                                "gazette_url": gazette_url,
                                "txt_url": txt_url,
                                "is_extra_edition": is_extra_edition
                            }
                            
                            classified_claims = generate_classified_claims(claims, metadata)
                            all_classified_claims.extend(classified_claims)
                
                logging.info(f"Finished processing gazette {idx}/{total_gazettes}")
            
            logging.info("Synthetic dataset created successfully.")
            return pd.DataFrame(all_classified_claims)
        else:
            logging.error("No data retrieved from the API.")
            return None

# Step 6: Generate and save the dataset
if __name__ == "__main__":
    logging.info("Running the synthetic dataset creation script.")
    
    all_synthetic_data = []  # List to accumulate all data

    for territory in territories:
        synthetic_dataset = create_synthetic_dataset(territory=territory, num_gazettes=1, num_claims_per_gazette=1, num_random_chunks=2)
        if synthetic_dataset is not None:
            all_synthetic_data.append(synthetic_dataset)  # Append the DataFrame to the list
            logging.info(f"Synthetic dataset for {territory['name']} created and appended.")
        else:
            logging.error(f"Failed to create synthetic dataset for {territory['name']}.")
    
    if all_synthetic_data is not None:
        combined_dataset = pd.concat(all_synthetic_data, ignore_index=True)
        combined_dataset.to_csv("synthetic_gazette_claims.csv", index=False)
        logging.info("Synthetic dataset created and saved as 'synthetic_gazette_claims.csv'.")
    else:
        logging.error("Failed to create synthetic dataset.")

2024-08-31 19:22:28,827 - INFO - Running the synthetic dataset creation script.
2024-08-31 19:22:28,828 - INFO - Starting synthetic dataset creation.
2024-08-31 19:22:28,829 - INFO - Querying gazettes with params: {'querystring': None, 'published_since': '2024-04-24', 'published_until': '2024-07-27', 'offset': 0, 'size': 1, 'territory_ids': [4302105]}
2024-08-31 19:22:29,262 - INFO - Gazettes retrieved successfully.
2024-08-31 19:22:29,263 - INFO - Total gazettes to process: 1
2024-08-31 19:22:29,263 - INFO - Processing gazette 1/1: https://querido-diario.nyc3.cdn.digitaloceanspaces.com/4302105/2024-05-29/8d9539d419d7bf7f0f0da7ed00ced1964fe50e41.txt
2024-08-31 19:22:29,263 - INFO - Downloading text from: https://querido-diario.nyc3.cdn.digitaloceanspaces.com/4302105/2024-05-29/8d9539d419d7bf7f0f0da7ed00ced1964fe50e41.txt


{'total_gazettes': 100, 'gazettes': [{'territory_id': '4302105', 'date': '2024-05-29', 'scraped_at': '2024-05-30T03:29:40.751442', 'url': 'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/4302105/2024-05-29/8d9539d419d7bf7f0f0da7ed00ced1964fe50e41.pdf', 'territory_name': 'Bento Gonçalves', 'state_code': 'RS', 'excerpts': [], 'edition': '2481', 'is_extra_edition': False, 'txt_url': 'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/4302105/2024-05-29/8d9539d419d7bf7f0f0da7ed00ced1964fe50e41.txt'}]}


2024-08-31 19:22:30,330 - INFO - Text downloaded successfully.
2024-08-31 19:22:30,387 - INFO - Text split into 283 chunks for gazette 1.
2024-08-31 19:22:30,387 - INFO - Gerando 1 afirmações a partir do texto do diário oficial.
2024-08-31 19:22:33,122 - INFO - Afirmações geradas pelo LLM: 1. Transferências da União e de suas Entidades: 2.872.273,00
Transferências dos Estados e do Distrito Federal e de suas Entidades: 7.410.000,00
Transferências de Instituições Privadas: 1.550.001,00
2024-08-31 19:22:33,122 - INFO - Gerando classificações para as afirmações.
2024-08-31 19:22:33,124 - INFO - Criando uma versão modificada da afirmação com a classificação: exaggerated
2024-08-31 19:22:33,543 - INFO - Afirmação modificada gerada: "Trezentas milhões de reais transferidos pela União e suas entidades em apenas um ano!"
2024-08-31 19:22:33,544 - INFO - Criando uma versão modificada da afirmação com a classificação: trustworthy-but
2024-08-31 19:22:34,280 - INFO - Afirmação modificada gerada: "

{'total_gazettes': 0, 'gazettes': []}


2024-08-31 19:22:45,156 - INFO - Gazettes retrieved successfully.
2024-08-31 19:22:45,157 - INFO - Total gazettes to process: 1
2024-08-31 19:22:45,157 - INFO - Processing gazette 1/1: https://querido-diario.nyc3.cdn.digitaloceanspaces.com/4303509/2023-09-25/1d8bd9aca86e49edf6ef7c26adb23baec6264170.txt
2024-08-31 19:22:45,157 - INFO - Downloading text from: https://querido-diario.nyc3.cdn.digitaloceanspaces.com/4303509/2023-09-25/1d8bd9aca86e49edf6ef7c26adb23baec6264170.txt


...

{'total_gazettes': 10, 'gazettes': [{'territory_id': '4318432', 'date': '2023-11-20', 'scraped_at': '2024-05-16T23:12:01.010062', 'url': 'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/4318432/2023-11-20/6e5150e85dbaecf2742692fe376c61f99c1a31d5.pdf', 'territory_name': 'São João do Polêsine', 'state_code': 'RS', 'excerpts': [], 'edition': '5', 'is_extra_edition': False, 'txt_url': 'https://querido-diario.nyc3.cdn.digitaloceanspaces.com/4318432/2023-11-20/6e5150e85dbaecf2742692fe376c61f99c1a31d5.txt'}]}


2024-08-31 19:24:01,211 - INFO - Text downloaded successfully.
2024-08-31 19:24:01,239 - INFO - Text split into 65 chunks for gazette 1.
2024-08-31 19:24:01,239 - INFO - Gerando 1 afirmações a partir do texto do diário oficial.
2024-08-31 19:24:02,898 - INFO - Afirmações geradas pelo LLM: Os fatos factuais potenciais do texto são:

MAURO ARAMI FLORES FAVERO é o contratado;
o valor da contratação é de R$ 17.000,00;
o objeto da contratação é a aquisição de uma estátua do Diácono João Luiz Pozzobon;
a estátua será fixada na praça Diácono João Luiz Pozzobon no centro de São João do Polêsine.
2024-08-31 19:24:02,899 - INFO - Gerando classificações para as afirmações.
2024-08-31 19:24:02,900 - INFO - Criando uma versão modificada da afirmação com a classificação: misleading
2024-08-31 19:24:03,550 - INFO - Afirmação modificada gerada: "Os fatos apresentados podem ser facilmente manipulados para sustentar uma visão parcial e enganadora sobre o assunto, levando a conclusões inverificáveis."
20

{'total_gazettes': 0, 'gazettes': []}


2024-08-31 19:24:12,631 - INFO - Gazettes retrieved successfully.
2024-08-31 19:24:12,631 - INFO - Total gazettes to process: 0
2024-08-31 19:24:12,631 - INFO - Synthetic dataset created successfully.
2024-08-31 19:24:12,632 - INFO - Synthetic dataset for Vera Cruz created and appended.
2024-08-31 19:24:12,636 - INFO - Synthetic dataset created and saved as 'synthetic_gazette_claims.csv'.


{'total_gazettes': 0, 'gazettes': []}
